# Generate Inference Data for Frontend Demo

In [1]:
from importlib import resources as impresources
from recurrent_health_events_prediction import configs
import yaml

import pandas as pd
import numpy as np

with open((impresources.files(configs) / 'data_config.yaml')) as f:
    config = yaml.safe_load(f)

In [ ]:
OUTPUT_DIR = 'C:/Users/User/projects/msc-thesis-recurrent-health-modeling/data/mimic-api-demo'

In [3]:
dataset_config = config['dataset']['mimic']
dataset_config['path'] = 'C:\\Users\\User\\projects\\msc-thesis-recurrent-health-modeling\\data\\mimic-iii-dataset'

inference_config = config['inference']['mimic']
required_cols = inference_config['required_cols']

preprocessing_config = config['training_data']['mimic']
patient_id_col = preprocessing_config['patient_id_col']
hosp_id_col = preprocessing_config['hosp_id_col']
time_col = preprocessing_config['time_col']

In [5]:
print("Required Columns for API\n")
for table in required_cols:
    print(f"Table: {table}")
    for col in required_cols[table]:
        print(f"  - {col}")

Required Columns for API

Table: admissions
  - HADM_ID
  - SUBJECT_ID
  - ADMITTIME
  - DISCHTIME
  - ADMISSION_TYPE
  - INSURANCE
  - ETHNICITY
  - DISCHARGE_LOCATION
Table: diagnoses
  - HADM_ID
  - SUBJECT_ID
  - ICD9_CODE
Table: icu_stays
  - SUBJECT_ID
  - HADM_ID
  - INTIME
  - OUTTIME
Table: prescriptions
  - HADM_ID
  - SUBJECT_ID
  - DRUG
Table: procedures
  - HADM_ID
  - SUBJECT_ID
  - ICD9_CODE
Table: patients
  - SUBJECT_ID
  - GENDER
  - DOB


In [6]:
base_dir = "C:/Users/User/projects/msc-thesis-recurrent-health-modeling/data/mimic-iii-preprocessed/copd_hf_renal_diabetes/multiple_hosp_patients"
train_data_path = base_dir + "/train_test/train_final.csv"
train_df = pd.read_csv(train_data_path)

test_data_path = base_dir + "/train_test/test.csv"
test_df = pd.read_csv(test_data_path)


In [7]:
train_df.head()

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,HOSPITALIZATION_DAYS,NUM_COMORBIDITIES,TYPES_COMORBIDITIES,HAS_DIABETES,HAS_COPD,HAS_CONGESTIVE_HF,...,ADMISSION_TYPE_URGENT,INSURANCE_GOVERNMENT,INSURANCE_MEDICAID,INSURANCE_MEDICARE,INSURANCE_PRIVATE,ETHNICITY_BLACK,ETHNICITY_HISPANIC,ETHNICITY_WHITE,DISCHARGE_LOCATION_HOME,DISCHARGE_LOCATION_POST_ACUTE_CARE
0,124,172461,2160-06-24 21:25:00,2160-07-15 15:10:00,20.739583,3,"['other', 'chronic_pulmonary_disease', 'cerebr...",False,True,False,...,False,False,False,False,True,False,False,True,True,False
1,124,112906,2161-12-17 03:39:00,2161-12-24 15:35:00,7.497222,3,"['other', 'peripheral_vascular_disease', 'chro...",False,True,False,...,False,False,False,True,False,False,False,True,True,False
2,124,134369,2165-05-21 21:02:00,2165-06-06 16:00:00,15.790278,4,"['other', 'peripheral_vascular_disease', 'chro...",False,True,False,...,False,False,False,True,False,False,False,True,False,True
3,138,108120,2131-10-31 08:00:00,2131-11-06 12:54:00,6.204167,4,"['other', 'congestive_heart_failure', 'myocard...",False,True,True,...,False,False,False,False,True,False,False,True,True,False
4,138,188284,2133-12-21 23:52:00,2134-01-11 13:00:00,20.547222,3,"['other', 'congestive_heart_failure', 'maligna...",False,False,True,...,False,False,False,False,True,False,False,True,True,False


In [9]:
subject_ids_train = train_df[patient_id_col].unique()
subject_ids_test = test_df[patient_id_col].unique()

print(f"Number of unique patients in train set: {len(subject_ids_train)}")
print(f"Number of unique patients in test set: {len(subject_ids_test)}")

Number of unique patients in train set: 1351
Number of unique patients in test set: 376


In [10]:
final_num_subjects = 1000
percentage_train = 0.7

num_train_subjects = int(final_num_subjects * percentage_train)
num_test_subjects = final_num_subjects - num_train_subjects

print(f"Number of subjects for inference demo: {final_num_subjects}")
print(f"  - Train subjects: {num_train_subjects}")
print(f"  - Test subjects: {num_test_subjects}")

Number of subjects for inference demo: 1000
  - Train subjects: 700
  - Test subjects: 300


In [11]:
np.random.seed(42)

demo_subjects_train = np.random.choice(subject_ids_train, size=num_train_subjects, replace=False)
demo_subjects_test = np.random.choice(subject_ids_test, size=num_test_subjects, replace=False)

demo_train_df = train_df[train_df[patient_id_col].isin(demo_subjects_train)]
demo_test_df = test_df[test_df[patient_id_col].isin(demo_subjects_test)]

demo_df = pd.concat([demo_train_df, demo_test_df], axis=0)

event_ids = demo_df[hosp_id_col].unique()

In [12]:
print("HADM_IDs in demo data:")
print(event_ids)

HADM_IDs in demo data:
[145243 105083 137006 ... 126023 151454 112748]


In [13]:
demo_df.head()

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,HOSPITALIZATION_DAYS,NUM_COMORBIDITIES,TYPES_COMORBIDITIES,HAS_DIABETES,HAS_COPD,HAS_CONGESTIVE_HF,...,ADMISSION_TYPE_URGENT,INSURANCE_GOVERNMENT,INSURANCE_MEDICAID,INSURANCE_MEDICARE,INSURANCE_PRIVATE,ETHNICITY_BLACK,ETHNICITY_HISPANIC,ETHNICITY_WHITE,DISCHARGE_LOCATION_HOME,DISCHARGE_LOCATION_POST_ACUTE_CARE
5,222,145243,2137-07-15 15:31:00,2137-07-17 12:00:00,1.853472,3,"['other', 'myocardial_infarct', 'cerebrovascul...",False,False,False,...,False,False,False,True,False,False,False,True,True,False
6,222,105083,2141-02-18 23:10:00,2141-02-21 15:50:00,2.694444,4,"['other', 'peripheral_vascular_disease', 'myoc...",False,False,False,...,False,False,False,True,False,False,False,True,True,False
7,222,137006,2142-06-11 07:15:00,2142-06-19 12:30:00,8.218750,7,"['myocardial_infarct', 'other', 'peripheral_va...",False,False,False,...,False,False,False,True,False,False,False,True,True,False
8,222,188038,2142-06-23 05:02:00,2142-07-01 17:30:00,8.519444,5,"['myocardial_infarct', 'other', 'malignant_can...",False,False,False,...,False,False,False,True,False,False,False,True,True,False
9,236,151459,2134-10-04 01:19:00,2134-10-15 16:00:00,11.611806,3,"['mild_liver_disease', 'malignant_cancer', 'ot...",False,False,False,...,False,False,False,False,True,False,False,False,True,False


In [14]:
target_cols = ['READMISSION_30_DAYS', 'DAYS_UNTIL_NEXT_HOSPITALIZATION', 'READMISSION_TIME_CAT']
target_df = demo_df.loc[demo_df['IS_LAST_EVENT'] == 1, [patient_id_col, hosp_id_col, *target_cols]]
target_df.head()

,SUBJECT_ID,HADM_ID,READMISSION_30_DAYS,DAYS_UNTIL_NEXT_HOSPITALIZATION,READMISSION_TIME_CAT
8,222,188038,0,229.743056,120+
10,236,182562,0,1361.902778,120+
15,291,125726,0,512.313194,120+
18,305,122211,0,766.170139,120+
21,323,128132,0,96.872222,30-120


In [15]:
subject_ids = demo_df[patient_id_col].unique()
print(f"Number of unique patients in demo data: {len(subject_ids)}")

print("Number of rows in target df:", target_df.shape[0])

Number of unique patients in demo data: 1000
Number of rows in target df: 1000


In [23]:
from recurrent_health_events_prediction.data_extraction.DataExtractor import DataExtractorMIMIC
from recurrent_health_events_prediction.data_extraction.data_types import DiseaseType

dataset_config = config['dataset']['mimic']

print(f"Dataset path: {dataset_config['path']}")

selected_diseases = preprocessing_config.get("selected_diseases", [DiseaseType.CHRONIC_PULMONARY_DISEASE,
                                                    DiseaseType.CONGESTIVE_HEART_FAILURE,
                                                    DiseaseType.DIABETES_WITH_COMPLICATION,
                                                    DiseaseType.RENAL_DISEASE])
selected_diseases = [DiseaseType(d) if isinstance(d, str) else d for d in selected_diseases]
data_extractor = DataExtractorMIMIC(dataset_config, selected_diseases)

data_extractor.load_data()
admissions_df = data_extractor.get_admissions_df()
icu_stays_df = data_extractor.get_icu_stays_df()
procedures_df = data_extractor.get_procedures_df()
prescriptions_df = data_extractor.get_prescriptions_df()
patients_metadata_df = data_extractor.get_patients_df()

Dataset path: C:\Users\User\projects\msc-thesis-recurrent-health-modeling\data\mimic-iii-dataset


C:\Users\User\projects\msc-thesis-recurrent-health-modeling\recurrent_health_events_prediction\data_extraction\DataExtractor.py:89: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  prescriptions_df = pd.read_csv(self.data_path + '/PRESCRIPTIONS.csv')


In [24]:
diagnoses_df = admissions_df[required_cols["diagnoses"]].copy()
admissions_df = admissions_df[required_cols["admissions"]].drop_duplicates().copy()
icu_stays_df = icu_stays_df[required_cols["icu_stays"]].copy()
procedures_df = procedures_df[required_cols["procedures"]].copy()
prescriptions_df = prescriptions_df[required_cols["prescriptions"]].copy()
patients_metadata_df = patients_metadata_df[required_cols["patients"]].copy()

In [25]:
admissions_selected_df = admissions_df[admissions_df[hosp_id_col].isin(event_ids)].copy()
diagnoses_selected_df = diagnoses_df[diagnoses_df[hosp_id_col].isin(event_ids)].copy()
icu_stays_selected_df = icu_stays_df[icu_stays_df[hosp_id_col].isin(event_ids)].copy()
procedures_selected_df = procedures_df[procedures_df[hosp_id_col].isin(event_ids)].copy()
prescriptions_selected_df = prescriptions_df[prescriptions_df[hosp_id_col].isin(event_ids)].copy()
patients_metadata_selected_df = patients_metadata_df[patients_metadata_df[patient_id_col].isin(subject_ids)].copy()

In [26]:
admissions_selected_df

,HADM_ID,SUBJECT_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,INSURANCE,ETHNICITY,DISCHARGE_LOCATION
839,122659,36,2131-05-12 19:49:00,2131-05-25 13:30:00,EMERGENCY,Medicare,WHITE,REHAB/DISTINCT PART HOSP
861,182104,36,2131-04-30 07:15:00,2131-05-08 14:00:00,EMERGENCY,Medicare,WHITE,HOME HEALTH CARE
1842,113649,291,2102-04-08 22:26:00,2102-04-12 13:00:00,EMERGENCY,Medicare,WHITE,HOME
1847,125726,291,2106-04-17 12:24:00,2106-04-19 15:10:00,EMERGENCY,Medicare,WHITE,HOME
1971,108015,305,2125-12-31 18:40:00,2126-01-10 11:30:00,EMERGENCY,Medicare,WHITE,SNF
...,...,...,...,...,...,...,...,...
633012,127852,97547,2125-10-29 03:22:00,2125-11-05 17:00:00,EMERGENCY,Medicare,WHITE,LONG TERM CARE HOSPITAL
633034,194938,97547,2125-10-18 13:00:00,2125-10-26 17:47:00,URGENT,Medicare,WHITE,REHAB/DISTINCT PART HOSP
633689,151454,99982,2156-11-28 11:56:00,2156-12-08 13:45:00,EMERGENCY,Medicare,WHITE,HOME HEALTH CARE
634132,152354,95561,2112-07-05 07:15:00,2112-07-22 16:45:00,ELECTIVE,Medicare,WHITE,LONG TERM CARE HOSPITAL


In [27]:
subject_ids_demo = admissions_selected_df['SUBJECT_ID'].nunique()
print(f"Number of unique patients in demo admissions diagnoses data: {subject_ids_demo}")

Number of unique patients in demo admissions diagnoses data: 1000


In [28]:
admissions_selected_df[time_col] = pd.to_datetime(admissions_selected_df[time_col])
icu_stays_selected_df['INTIME'] = pd.to_datetime(icu_stays_selected_df['INTIME'])

admissions_selected_df.sort_values(by=[patient_id_col, time_col], inplace=True)
diagnoses_selected_df.sort_values(by=[patient_id_col], inplace=True)
icu_stays_selected_df.sort_values(by=[patient_id_col, 'INTIME'], inplace=True)
procedures_selected_df.sort_values(by=[patient_id_col], inplace=True)
prescriptions_selected_df.sort_values(by=[patient_id_col], inplace=True)
patients_metadata_selected_df.sort_values(by=[patient_id_col], inplace=True)
target_df.sort_values(by=[patient_id_col], inplace=True)

In [29]:
admissions_selected_df.to_csv(f"{OUTPUT_DIR}/admissions.csv", index=False)
icu_stays_selected_df.to_csv(f"{OUTPUT_DIR}/icu_stays.csv", index=False)
procedures_selected_df.to_csv(f"{OUTPUT_DIR}/procedures.csv", index=False)
prescriptions_selected_df.to_csv(f"{OUTPUT_DIR}/prescriptions.csv", index=False)
patients_metadata_selected_df.to_csv(f"{OUTPUT_DIR}/patients.csv", index=False)
diagnoses_selected_df.to_csv(f"{OUTPUT_DIR}/diagnoses.csv", index=False)
target_df.to_csv(f"{OUTPUT_DIR}/target.csv", index=False)